In [10]:
!pip install langchain langchain-community openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.9 MB/s eta 0:00:00


In [11]:
import os
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
os.environ['OPENAI_API_KEY'] = "여러분들의 Key 값"

In [12]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
query_result = embeddings.embed_query('저는 배가 고파요')
print(query_result)

[-0.016628302994777876, -0.021754816646823914, 0.015221799122960466, -0.027315110561221212, -0.036805731112148424, 0.011810696008922395, -0.034571097051648625, -0.006763053273465178, -0.023950014783368005, -0.016825477026620846, -0.00816298573305343, 0.010877408323411957, -0.010601365237625358, -0.02259608965996462, 0.011258610768671792, -0.004909621658225213, 0.012106457056686847, -0.0028935219789422272, 0.008084115747787205, -0.01616823149557441, 0.001459084366461299, -0.01459084389744364, 0.018205691524231245, -0.012724267595100168, 0.003319088208837693, 0.00636541996932596, 0.005356548470325174, -0.019099545148431162, -0.009161997785403993, -0.0017252687040896196, 0.034492228929027584, -0.016536290185053336, 6.315715427958754e-05, 0.0030939817168997743, 0.0073874361554302536, -0.0055405768837420415, -0.006371992312877713, 0.0034143888201526515, -0.00385474305587021, -0.0021738384856621936, -0.022425206864973853, -0.010312177464735253, 0.011357196853270681, -0.02467298561257716, 0.0

In [13]:
data = ['저는 배가 고파요',
        '저기 배가 지나가네요',
        '굶어서 허기가 지네요',
        '허기 워기라는 게임이 있는데 즐거워',
        '스팀에서 재밌는 거 해야지',
        '스팀에어프라이어로 연어구이 해먹을거야']

df = pd.DataFrame(data, columns=['text'])
df

,text
0,저는 배가 고파요
1,저기 배가 지나가네요
2,굶어서 허기가 지네요
3,허기 워기라는 게임이 있는데 즐거워
4,스팀에서 재밌는 거 해야지
5,스팀에어프라이어로 연어구이 해먹을거야


In [14]:
def get_embedding(text):
  return embeddings.embed_query(text)

df['embedding'] = df.apply(lambda row: get_embedding(
        row.text,
    ), axis=1)
df

,text,embedding
0,저는 배가 고파요,"[-0.016628302994777876, -0.021754816646823914,..."
1,저기 배가 지나가네요,"[-0.0033146261427528688, -0.027557911195226485..."
2,굶어서 허기가 지네요,"[-0.006115510937439029, -0.0070621542151335275..."
3,허기 워기라는 게임이 있는데 즐거워,"[-0.011373619934562202, -0.011753614427933356,..."
4,스팀에서 재밌는 거 해야지,"[-0.016117957883498426, -0.014398175804554103,..."
5,스팀에어프라이어로 연어구이 해먹을거야,"[-0.002163747705450216, -0.03004803712770594, ..."


In [15]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_answer_candidate(df, query):
    query_embedding = get_embedding(
        query
    )
    df["similarity"] = df.embedding.apply(lambda x: cos_sim(np.array(x),
                                                            np.array(query_embedding)))
    top_three_doc = df.sort_values("similarity",
                                ascending=False).head(3)
    return top_three_doc

In [16]:
sim_result = return_answer_candidate(df, '아무것도 안 먹었더니 꼬르륵 소리가 나네')
sim_result

,text,embedding,similarity
2,굶어서 허기가 지네요,"[-0.006115510937439029, -0.0070621542151335275...",0.836827
5,스팀에어프라이어로 연어구이 해먹을거야,"[-0.002163747705450216, -0.03004803712770594, ...",0.815895
0,저는 배가 고파요,"[-0.016628302994777876, -0.021754816646823914,...",0.812362
